# Phase 5: Report & Documentation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

## Segment Profile Visualization 

In [ ]:
# Load data from previous phases
try:
    df = pd.read_csv('customers_with_predictions.csv')  # From Phase 4
    print("Loaded columns:", df.columns.tolist())  # Debug: Show available columns
    
    # Check for required columns and create if missing
    if 'Response' not in df.columns:
        print("Warning: 'Response' column not found - using predicted probabilities")
        df['Response'] = (df['Response_Probability'] > 0.5).astype(int)
        
    if 'Cluster' not in df.columns:
        raise ValueError("Cluster column missing - run Phase 3 first")
        
except FileNotFoundError:
    raise FileNotFoundError("Run Phase 4 first to generate customers_with_predictions.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Load data
try:
    df = pd.read_csv('customers_with_predictions.csv')  # From Phase 4
    print("Loaded columns:", df.columns.tolist())

    # Create 'Response' if missing
    if 'Response' not in df.columns:
        print("Warning: 'Response' column not found - using predicted probabilities")
        df['Response'] = (df['Response_Probability'] > 0.5).astype(int)

    if 'Cluster' not in df.columns:
        raise ValueError("Cluster column missing - run Phase 3 first")

except FileNotFoundError:
    raise FileNotFoundError("Run Phase 4 first to generate customers_with_predictions.csv")

# Assign segment labels
df['Segment'] = df['Cluster'].apply(lambda x: f'Segment {x+1}')

# Format currency for plots
def currency_formatter(x, pos):
    return f'${x:,.0f}'

currency_format = FuncFormatter(currency_formatter)

# Features to include in the profile plot
profile_features = [
    'Income', 'Total_Spend', 'Recency',
    'Family_Size', 'MntWines', 'NumWebVisitsMonth'
]

# Create enhanced profile plot
plt.figure(figsize=(14, 8))
for i, feature in enumerate(profile_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(
        x='Segment',
        y=feature,
        data=df,
        order=sorted(df['Segment'].unique()),
        hue='Segment',  # Add this to avoid FutureWarning
        legend=False
    )
    plt.title(feature.replace('_', ' '))
    if feature in ['Income', 'Total_Spend', 'MntWines']:
        plt.gca().yaxis.set_major_formatter(currency_format)
    plt.xticks(rotation=45)

plt.suptitle('Customer Segment Profiles', y=1.02, fontsize=16)
plt.tight_layout()
plt.savefig('enhanced_segment_profiles.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Segment profile plot saved as 'enhanced_segment_profiles.png'")


## Response Analysis 

In [ ]:
# Merge with segment stats
response_analysis = df.groupby('Segment').agg({
    'Response_Probability': ['mean', 'count'],
    'Response': 'sum'
}).reset_index()
response_analysis.columns = ['Segment', 'Avg_Prob', 'Count', 'Actual_Responses']

# Calculate conversion rate and uplift
response_analysis['Conversion_Rate'] = response_analysis['Actual_Responses'] / response_analysis['Count']
response_analysis['Uplift'] = response_analysis['Avg_Prob'] / response_analysis['Avg_Prob'].mean() - 1

# Visualize
plt.figure(figsize=(12, 6))
sns.barplot(x='Segment', y='Uplift', data=response_analysis, 
            order=sorted(df['Segment'].unique()))
plt.axhline(0, color='black', linestyle='--')
plt.title('Campaign Response Uplift by Segment', fontsize=14)
plt.ylabel('Uplift vs Average (%)')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.0%}'))
plt.savefig('response_uplift.png', dpi=300, bbox_inches='tight')
plt.close()

## Generate Recommendations

In [ ]:
def generate_segment_recommendations(segment):
    seg_data = df[df['Segment'] == segment].iloc[0]
    recs = []
    
    # Spending behavior
    if seg_data['Total_Spend'] > df['Total_Spend'].median():
        recs.append("Premium customer: Target with exclusive offers")
    else:
        recs.append("Value customer: Focus on price-sensitive promotions")
    
    # Engagement
    if seg_data['Recency'] < df['Recency'].median():
        recs.append("Active buyer: Frequent communication")
    else:
        recs.append("Lapsed customer: Win-back campaign needed")
    
    # Product preferences
    if seg_data['MntWines'] > df['MntWines'].median():
        recs.append("Wine enthusiast: Cross-sell accessories")
    
    # Response probability
    prob = response_analysis[response_analysis['Segment'] == segment]['Avg_Prob'].values[0]
    if prob > 0.5:
        recs.append(f"High response likelihood ({prob:.0%}): Prioritize in campaigns")
    else:
        recs.append(f"Low response likelihood ({prob:.0%}): Test alternative approaches")
    
    return recs

## Create Final Report 

In [ ]:
report_content = []
report_content.append("=== CUSTOMER SEGMENTATION INSIGHTS ===")
report_content.append("\n1. SEGMENT PROFILES\n")

# Add segment statistics
for segment in sorted(df['Segment'].unique()):
    seg_data = df[df['Segment'] == segment].iloc[0]
    report_content.append(
        f"{segment} (n={len(df[df['Segment'] == segment])}):\n"
        f"- Median Income: ${seg_data['Income']:,.0f}\n"
        f"- Median Spend: ${seg_data['Total_Spend']:,.0f}\n"
        f"- {'Frequent' if seg_data['Recency'] < df['Recency'].median() else 'Infrequent'} buyers\n"
        f"- {'Wine lovers' if seg_data['MntWines'] > df['MntWines'].median() else 'Casual drinkers'}\n"
    )

# Add recommendations
report_content.append("\n2. ACTIONABLE RECOMMENDATIONS\n")
for segment in sorted(df['Segment'].unique()):
    report_content.append(f"{segment}:")
    for i, rec in enumerate(generate_segment_recommendations(segment), 1):
        report_content.append(f"  {i}. {rec}")
    report_content.append("")

# Add response analysis
report_content.append("\n3. CAMPAIGN OPTIMIZATION\n")
report_content.append("Segment Response Potential:")
report_content.append(response_analysis.to_string(index=False))

# Save reportreport_content = []
report_content.append("=== CUSTOMER SEGMENTATION INSIGHTS ===")
report_content.append("\n1. SEGMENT PROFILES\n")

# Add segment statistics
for segment in sorted(df['Segment'].unique()):
    seg_data = df[df['Segment'] == segment].iloc[0]
    report_content.append(
        f"{segment} (n={len(df[df['Segment'] == segment])}):\n"
        f"- Median Income: ${seg_data['Income']:,.0f}\n"
        f"- Median Spend: ${seg_data['Total_Spend']:,.0f}\n"
        f"- {'Frequent' if seg_data['Recency'] < df['Recency'].median() else 'Infrequent'} buyers\n"
        f"- {'Wine lovers' if seg_data['MntWines'] > df['MntWines'].median() else 'Casual drinkers'}\n"
    )
with open('executive_report.txt', 'w') as f:
    f.write("\n".join(report_content))

## Save Final Outputs 

In [ ]:
# Save enriched data
df.to_csv('final_customer_data.csv', index=False)

print("Phase 5 completed successfully!")
print("Key outputs generated:")
print("- enhanced_segment_profiles.png: Detailed segment characteristics")
print("- response_uplift.png: Campaign performance by segment")
print("- executive_report.txt: Business-ready recommendations")
print("- final_customer_data.csv: Complete analysis dataset")